**Test for Python Developer – API Data Extraction role**

1. Uses Youtube to get channel info and other related data

In [2]:

import requests
from bs4 import BeautifulSoup
import os
import time
import re
import pandas as pd  
import urllib
from PIL import Image

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait

from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException

import xlsxwriter


**step 1 :** Scrape channel info from videos avalaible on youtube homepage

In [3]:
PROJECT_DIR = os.getcwd()
print(PROJECT_DIR)
DATA_DIR = os.path.join(PROJECT_DIR,'ScrappedData')
YT_DATA_DIR = os.path.join(DATA_DIR,'YoutubeData')

c:\Users\user\Desktop\data_scrape_test


In [4]:
if not os.path.exists(PROJECT_DIR):
  os.mkdir(PROJECT_DIR)

if not os.path.exists(DATA_DIR):
  os.mkdir(DATA_DIR)

if not os.path.exists(YT_DATA_DIR):
  os.mkdir(YT_DATA_DIR)

In [ ]:
# !pip install webdriver-manager

In [5]:
custom_path=r'c:\Users\user\Desktop\data_scrape_test'

driver = webdriver.Chrome(ChromeDriverManager(path=custom_path).install())




====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [c:\Users\user\Desktop\data_scrape_test\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [ ]:

#xlsx
# !pip install xlsxwriter==3.0.1 


In [6]:
def get_channel_url_and_id(driver):
    channel_names = []
    urls = []
    channel_name_elements = driver.find_elements_by_class_name(name='yt-formatted-string')

    for el in channel_name_elements:
        c_name = el.get_property('text')
        channel_names.append(c_name)
        url = el.get_property('href')
        urls.append(url)

    return channel_names,urls


In [7]:
def get_channel_info(driver):
    channel_names = []
    urls = []
    channel_name_elements = driver.find_elements_by_class_name(name='yt-formatted-string')

    for el in channel_name_elements:
        c_name = el.get_property('text')
        channel_names.append(c_name)
        url = el.get_property('href')
        urls.append(url)

    return channel_names,urls

def get_subscriber_count(text):
    count = text.split(' subscribers')[0]
    if 'K' in count:
        subscribers = count.split('K')[0]
        subscribers = int(float(subscribers)*1000)
    elif 'M' in count:
        subscribers = count.split('M')[0]
        subscribers = int(float(subscribers)*1000000)
    else:
        subscribers = int(count)
    return subscribers


In [8]:
yt_url = 'https://youtube.com'


In [ ]:
#gets channel name and url from yt Home page

timeout = 5
channel_names = []
channel_urls = []
driver.get(yt_url)

##scroll page to get new contnet

#scroll down 2 times
#increase the range to sroll more
n_scrolls = 40
for j in range(0, n_scrolls):
    to_loc = str(j*2000)
    arg = "window.scrollTo(0,{});".format(to_loc)
    driver.execute_script(arg)
    time.sleep(5)

try:
    element_present = EC.presence_of_element_located((By.ID, 'content'))
    WebDriverWait(driver, timeout).until(element_present)
    c_names,c_urls = get_channel_url_and_id(driver)
    channel_names.extend(c_names)
    channel_urls.extend(c_urls)
except TimeoutException:
    print ("Timed out waiting for page to load")

In [ ]:
#save scraped channel info and channel urls as csv

dict = {'channel_name': channel_names, 'channel_url': channel_urls}  

yt_info = pd.DataFrame(dict) 
    
# saving the dataframe 
yt_info.to_csv(os.path.join(DATA_DIR,'yt_users_data.csv'))

Step 1 complete
saved channel urls and channel names as a csv file

In [9]:
#read the saved csv file to get yt channel urls
yt_info = pd.read_csv(os.path.join(DATA_DIR,'yt_users_data.csv'))

channel_urls = yt_info['channel_url']


In [10]:
yt_info.head()

,Unnamed: 0,channel_name,channel_url
0,0,Marley Dockery,https://www.youtube.com/user/MrCornholelio
1,1,Oggy Hindi - हिन्दी,https://www.youtube.com/channel/UC9TgvYNUsMBPl...
2,2,Quantum Tech HD,https://www.youtube.com/c/QuantumTech
3,3,PRANAV YT FF,https://www.youtube.com/channel/UCWxmHsZD62sHU...
4,4,HouseMusicHD,https://www.youtube.com/c/FutureHouseHD


In [12]:
def get_image(url,name):
   urllib.request.urlretrieve(url, name)
   i = Image.open(name)
   i.thumbnail((100, 100), Image.LANCZOS)
   i.save(name)

def get_videos_info(driver,n=12):
    timeout = 5
    video_urls = []
    video_durations = []
    video_thumbnails = []
    video_titles = []
    channel_urls = yt_info['channel_url']
    driver.get(channel_urls[0]+"/videos")
    ##scroll page to get new contnet
    #increase the range to sroll more
    n_scrolls = 2
    for j in range(0, n_scrolls):
        to_loc = str(j*2000)
        arg = "window.scrollTo(0,{});".format(to_loc)
        driver.execute_script(arg)
        time.sleep(5)
    try:
        element_present = EC.presence_of_element_located((By.ID, 'thumbnail'))
        WebDriverWait(driver, timeout).until(element_present)
        video_links = driver.find_elements_by_id('thumbnail')[:n]
        duration_list = driver.find_elements_by_xpath("//span[@class='style-scope ytd-thumbnail-overlay-time-status-renderer']")[:n]
        thumbnails_list = driver.find_elements_by_id('img')[1:n+1]
        video_titles = driver.find_elements_by_id('video-title')[:n]
        views_list = []
        premiered = []
        likes = []
        dislikes = []
        descriptions = []
        comments_list = []
        hashtags_list = []

        ret = {'links':[x.get_attribute('href') for x in video_links],'durations':[x.text for x in duration_list],'thumbnails':[x.get_attribute('src') for x in thumbnails_list],'titles': [x.text for x in video_titles] }
        for link in ret['links']:
            try:
                driver.get(link)
                element_present = EC.presence_of_element_located((By.ID, 'count'))
                WebDriverWait(driver, timeout).until(element_present)
                arg = "window.scrollTo(0,{});".format(700)
                driver.execute_script(arg)
                time.sleep(4)
                #get video stats
                views = driver.find_elements_by_id('count')[1].text.split(' views')[0]
                views_list.append(views)
                try:
                    premere_date = driver.find_elements_by_xpath("//yt-formatted-string[@class='style-scope ytd-video-primary-info-renderer']")[1].text.split('Premiered ')[1]
                except:
                    premere_date = 'NA'
                premiered.append(premere_date)
                num_likes = driver.find_elements_by_xpath("//yt-formatted-string[@class='style-scope ytd-toggle-button-renderer style-text']")[2].text
                num_dis_likes = driver.find_elements_by_xpath("//yt-formatted-string[@class='style-scope ytd-toggle-button-renderer style-text']")[3].text
                likes.append(num_likes)
                dislikes.append(num_dis_likes)
                desc = driver.find_elements_by_id('description')[1].text
                descriptions.append(desc)
                hashtags = regex = re.findall("#(\w+)", desc)
                hashtags = "#"+",#".join(hashtags)
                hashtags_list.append(hashtags)
                try:
                    num_comments = driver.find_elements_by_id('count')[2].text.split(' Comments')[0]
                except:
                    num_comments = "NA"
                comments_list.append(num_comments)

            except TimeoutException:
                print ("Timed out waiting for page to load")


    except TimeoutException:
        print ("Timed out waiting for page to load")
        

    return {'links':ret['links'],'durations':ret['durations'],'thumbnails':ret['thumbnails'],
    'titles': ret['titles'],'views':views_list,'premiere_date':premiered,'likes':likes,'dislikes':dislikes,'description':descriptions,'hashtags':hashtags_list,'comments':comments_list }

In [13]:
def remove_png_images():
    print("Removing Old images")
    images = os.listdir(PROJECT_DIR)
    for image in images:
        if image.endswith(".png"):
            os.remove(os.path.join(PROJECT_DIR, image))

In [16]:
#gets channel stats and downloads dp as dp.png of channel
#cretates a xls file name as channel name
# and saves the stats obtained
#and gets stats of few videos of channel

def get_channel_details(channel_url):
    about_page_url = channel_url+'/about'
    try:
        driver.get(about_page_url)
        element_present = EC.presence_of_element_located((By.ID, 'right-column'))
        WebDriverWait(driver, timeout).until(element_present)
        channel_name = driver.find_element_by_class_name('style-scope ytd-channel-name').text
        
        try:
            verified_status = driver.find_element_by_class_name('badge-style-type-verified').get_attribute('aria-label')
        except NoSuchElementException:
            verified_status = 'Not-Verified'
    
        try:
            dp_url = driver.find_element_by_id('img').get_attribute('src')
            get_image(dp_url,'dp.png')
        except:
            print("DP not found")
        #to download image using dp_url

        try:
            stats = driver.find_element_by_id('right-column').text.split('\n')
            joined = stats[1].split('Joined')[1]
            views = stats[2].split('views')[0]

        except NoSuchElementException:
            joined =  'NA'
            views = 'NA'

        description = driver.find_element_by_id('description').text
        email_ids = re.findall('\S+@\S+', description)
        email_ids_str =  ",".join(email_ids)
        soup = BeautifulSoup(driver.page_source,'lxml')
        links_container = soup.find(id="link-list-container")
        links_url_list = []
        links_url = ' '
        for link in links_container.findChildren('a'):
            link_url = link.get('href')
            links_url_list.append(link_url.split('&q=')[1])
            links_url = ','.join(links_url_list)
       
        workbook = xlsxwriter.Workbook(str(YT_DATA_DIR)+'/yt_'+str(channel_name)+'.xlsx')
        worksheet = workbook.add_worksheet()
        worksheet.write('A1', 'Name')
        worksheet.write('B1', 'Url')
        worksheet.write('C1', 'Subscribers')
        worksheet.write('D1', 'Verified')
        worksheet.write('E1', 'description')
        worksheet.write('F1', 'email')
        worksheet.write('G1', 'joined')
        worksheet.write('H1', 'Total views')
        worksheet.write('I1', 'External Links')
        worksheet.write('J1', 'Profile Picture url')
        worksheet.write('K1', 'Profile Picture')

        worksheet.set_row(11,70)
        worksheet.set_row(1,50)
        worksheet.set_column_pixels(11,2,80)

        worksheet.write('A2', channel_name)
        worksheet.write('B2', channel_url)
        worksheet.write('C2', num_subscribers)
        worksheet.write('D2', verified_status)
        worksheet.write('E2', description)
        worksheet.write('F2', email_ids_str)
        worksheet.write('G2', joined)
        worksheet.write('H2', views)
        worksheet.write('I2', links_url)
        worksheet.write('J2', dp_url)
        worksheet.insert_image('K2','dp.png')

        #call another function to save post details in a loop
        worksheet.set_column('A:A', 30)

    #save recent postsheading

        worksheet.write('A11', 'Thumbnail')
        worksheet.write('B11', 'Description')
        worksheet.write('C11', 'Title')
        worksheet.write('D11', 'Duration')
        worksheet.write('E11', 'Views')
        worksheet.write('F11', 'likes')
        worksheet.write('G11', 'Dislikes')
        worksheet.write('H11', 'Premiered')
        worksheet.write('I11', 'Hashtags')
        worksheet.write('J11', 'Comments')
        worksheet.write('K11', 'link')
        posts_info = get_videos_info(driver,6)
        row = 12
        for im in posts_info['thumbnails']:
            if im != None:
                get_image(im, str(row)+'.png')
                worksheet.insert_image('A'+str(row), str(row)+'.png')
            else:
                worksheet.write_row(0,12,'NA')

            row = row +1
    
        worksheet.write_column(12,1,posts_info['description'])
        worksheet.write_column(12,2,posts_info['titles'])
        worksheet.write_column(12,3,posts_info['durations'])
        worksheet.write_column(12,4,posts_info['views'])
        worksheet.write_column(12,5,posts_info['likes'])
        worksheet.write_column(12,6,posts_info['dislikes'])
        worksheet.write_column(12,7,posts_info['premiere_date'])
        worksheet.write_column(12,8,posts_info['hashtags'])
        worksheet.write_column(12,9,posts_info['comments'])
        worksheet.write_column(12,10,posts_info['links'])


        workbook.close()
        #remove all png images
        remove_png_images()
        print("File Saved as "+'yt_'+str(channel_name)+'.xlsx')
    except TimeoutException:
        print ("Timed out waiting for page to load")


In [17]:
timeout = 6
#first checks subscriber count
for channel_url in channel_urls[:11]:
    try:
        driver.get(channel_url)
        element_present = EC.presence_of_element_located((By.ID, 'thumbnail'))
        WebDriverWait(driver, timeout).until(element_present)
        subscriber_count_element = driver.find_element_by_id('subscriber-count')
        try:
            num_subscribers = get_subscriber_count(subscriber_count_element.text)
            if(num_subscribers > 10000):
                print("Subscribers : {} Channel : {}".format(num_subscribers,channel_url))
                get_channel_details(channel_url)
        except:
            print('Skipping as no subscriber infofound')

    except TimeoutException:
        print ("Timed out waiting for page to load")

Subscribers : 82200 Channel : https://www.youtube.com/user/MrCornholelio
Removing Old images
File Saved as yt_Marley Dockery.xlsx
Subscribers : 6900000 Channel : https://www.youtube.com/channel/UC9TgvYNUsMBPl6J2xbL-4qg
Timed out waiting for page to load
Removing Old images
File Saved as yt_Oggy Hindi - हिन्दी.xlsx
Subscribers : 9620000 Channel : https://www.youtube.com/c/QuantumTech
Timed out waiting for page to load
Removing Old images
File Saved as yt_Quantum Tech HD.xlsx
Skipping as no subscriber infofound
Subscribers : 6040000 Channel : https://www.youtube.com/c/FutureHouseHD
Timed out waiting for page to load
Removing Old images
File Saved as yt_HouseMusicHD.xlsx
Subscribers : 3340000 Channel : https://www.youtube.com/c/Inventor101
Removing Old images
File Saved as yt_Inventor 101.xlsx
Subscribers : 2089999 Channel : https://www.youtube.com/c/WhatTheFukrey
Skipping as no subscriber infofound
Subscribers : 6210000 Channel : https://www.youtube.com/c/UjjwalGamer
Skipping as no subsc